# Extract fields from a human message using the Inflection AI API

This notebook demonstrates how to extract fields from a human message using the Inflection AI API.

Think of the "Instruction" as similar to a "System Prompt" in other LLM applications.

The "Human" message is the message you want to extract fields from. This typically is a message from a user of your application.

- We are using the Productivity model `inflection_3_productivity` in this example as opposed to the `inflection_3_pi` model. 
- We've provided an example `fetch` function that we utilize to make an asynchronous POST request to the Inflection AI API.
- We're using the `parse_xml_response` function to parse the XML response from the Inflection AI API. We've found best results using this approach to get consistent results since there is not a "guaranteed JSON" mode for the Inflection AI API (yet!).




In [ ]:
import os
import time
import logging
import aiohttp
from datetime import datetime
import re

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Set your Inflection API key in your environment variables
inflection_api_key = os.getenv("INFLECTION_API_KEY")

In [ ]:
async def fetch(context: list[dict], model: str = "inflection_3_pi"):
    base_url = "https://layercake.pubwestus3.inf7ks8.com/external/api/inference"

    headers = {
        "Authorization": f"Bearer {inflection_api_key}",
        "Content-Type": "application/json",
    }

    json_payload = {"config": model, "context": context}

    try:
        async with aiohttp.ClientSession() as session:
            start_time = time.time()
            async with session.post(
                base_url, headers=headers, json=json_payload
            ) as response:
                response.raise_for_status()
                chat_completion = await response.json()
                end_time = time.time()
                duration = (end_time - start_time) * 1000  # Convert to milliseconds
                logger.info(f"Inflection API request took {duration:.2f} ms")
                return chat_completion.get("text", None)
    except Exception as e:
        logger.error(f"Error occurred: {str(e)}")
        return None

In [ ]:
system_instruction_prompt = """
    You are a helpful AI assistant designed to assist users in scheduling meetings.

    # Your Purpose
    1. You will be given a request from a user to schedule a meeting.
    2. Your job is to extract the parts of the request that are relevant to scheduling the meeting.
    3. Use Local Date: as a reference only when a specific date is not provided in the user's request.

    # Valid Parts
    - Time Duration: The duration of the meeting in minutes.
    - Date: The date of the meeting.
    - Participants: List of participants of the meeting.
    - Meeting Topic: The main topic or subject of the meeting.
    - Additional Details: Any additional context or details mentioned.

    # Example Input
    Schedule 30 minutes on Friday with Masha and Matt to talk about Product.
    Schedule half an hour on Friday with Masha and Matt to talk about Product.

    # Example Output
    <time_duration>30</time_duration>
    <date>2024-08-09</date>
    <participants>
    <participant>Masha</participant>
    <participant>Matt</participant>
    </participants>
    <meeting_topic>Product</meeting_topic>
    <additional_details>talk about Product</additional_details>

    # Notes on the date:
    - The date should be in the format "YYYY-MM-DD".
    - If a specific date is provided in the user's request (e.g., "09/13/2024"), use that date and convert it to the YYYY-MM-DD format.
    - If the date is not provided, return today's date.
    - If the date just says "Friday," calculate this week's Friday from the current date.
    - If the date says "next Friday," calculate next week's Friday from the current date.
    - Ensure that the calculated date is in the future. If "Friday" refers to a day in the past based on the current date, assume it refers to the next occurrence of that day.
    - Always return a future date for relative terms like "Friday" or "next Friday."

    # Notes on time duration:
    - The time duration should be in minutes.
    - The user might say "30 minutes" or "half an hour," which should be converted to minutes.
    - The user might say "1 hour," which should be converted to 60 minutes.
    - If a specific time range is provided (e.g., "10:00am-10:30am"), calculate the duration in minutes.
    - If the time duration is not provided, assume it is 30 minutes.

    # Output Format - XML
    You will respond using XML tags for each extracted piece of information. You don't need to provide explanation or any other information, just return the extracted parts within the appropriate XML tags.

    # Format of the Output
    <time_duration>extracted_time_duration</time_duration>
    <date>extracted_date</date>
    <participants>
    <participant>participant1</participant>
    <participant>participant2</participant>
    </participants>
    <meeting_topic>extracted_meeting_topic</meeting_topic>
    <additional_details>extracted_additional_details</additional_details>
"""

In [ ]:
def parse_xml_response(xml_string):
    result = {}
    result["time_duration"] = int(
        re.search(r"<time_duration>(\d+)</time_duration>", xml_string).group(1)
    )
    result["date"] = re.search(r"<date>(.+?)</date>", xml_string).group(1)
    result["participants"] = re.findall(r"<participant>(.+?)</participant>", xml_string)
    result["meeting_topic"] = re.search(
        r"<meeting_topic>(.+?)</meeting_topic>", xml_string
    ).group(1)
    result["additional_details"] = re.search(
        r"<additional_details>(.+?)</additional_details>", xml_string
    ).group(1)
    return result

async def extract_meeting_invite_parts(users_ask, date):
    context = [
        {"type": "Instruction", "text": system_instruction_prompt},
        {
            "type": "Human",
            "text": f"Schedule meeting request: {users_ask} Today's date: {date}",
        },
    ]
    result = await fetch(context, model="inflection_3_productivity")
    return parse_xml_response(result)

## Test Scenario: Basic Meeting Extraction

In [ ]:
async def test_extract_meeting_invite_parts_basic_scenario():
    """
    Scenario: The user provides straightforward info (a single participant,
    direct date, direct time duration and a topic) to schedule a meeting.
    Expected Outcome: correct information in every component.
    """
    print("Starting test: test_extract_meeting_invite_parts_basic_scenario")

    user_message = "Schedule a meeting with Matt on 09/13/2024 for 10:00am-10:30am to discuss Products."
    today = datetime.now().strftime("%Y-%m-%d")

    expected_output = {
        "time_duration": 30,
        "date": "2024-09-13",
        "participants": ["Matt"],
        "meeting_topic": "Products",
        "additional_details": "discuss Products",
    }

    extracted_parts = await extract_meeting_invite_parts(
        users_ask=user_message, date=today
    )
    print(f"Extracted parts: {extracted_parts}")

    assert (
        expected_output == extracted_parts
    ), f"Extracted parts do not match the expected output. Expected: {expected_output}, Got: {extracted_parts}"

    print("Test completed successfully!")

# Run the test
await test_extract_meeting_invite_parts_basic_scenario()